In [15]:
import pandas as pd
from collections import defaultdict

In [16]:
gtf_file = "/Users/kandarpjoshi/ref/chm13v2.0_RefSeq_Liftoff_v5.2.gff3"

In [33]:
def parse_gtf(gtf_file):
    transcripts = defaultdict(lambda: {
        "exons": [],
        "cds": []
    })

    with open(gtf_file, 'r') as f:
        for line in f:
            if line.startswith("#"):
                continue
            fields = line.strip().split('\t')
            feature_type = fields[2]
            chrom = fields[0]
            start = int(fields[3])
            end = int(fields[4])
            strand = fields[6]
            attributes = fields[8]

            # Extract transcript_id and gene_id
            attr_dict = {
                item.split('=')[0]: item.split('=')[1]
                for item in attributes.split(';') if '=' in item
            }
            transcript_id = attr_dict.get("transcript_id")

            if feature_type == "exon" and transcript_id:
                transcripts[transcript_id]["exons"].append((chrom, start, end, strand))
            elif feature_type == "CDS" and transcript_id:
                transcripts[transcript_id]["cds"].append((chrom, start, end, strand))

    regions = {
        "5UTR": [],
        "CDS": [],
        "3UTR": []
    }

    # Split regions into 5' UTR, CDS, and 3' UTR
    for transcript_id, features in transcripts.items():
        exons = sorted(features["exons"], key=lambda x: x[1])  # Sort by start position
        cds = sorted(features["cds"], key=lambda x: x[1])      # Sort by start position

        if not exons or not cds:
            continue

        chrom, strand = exons[0][0], exons[0][3]

        if strand == "+":
            five_utr = [(chrom, exon[1], min(exon[2], cds[0][1] - 1), strand) for exon in exons if exon[2] < cds[0][1]]
            cds_region = [(chrom, max(exon[1], cds[0][1]), min(exon[2], cds[-1][2]), strand) for exon in exons if exon[1] <= cds[-1][2] and exon[2] >= cds[0][1]]
            three_utr = [(chrom, max(exon[1], cds[-1][2] + 1), exon[2], strand) for exon in exons if exon[1] > cds[-1][2]]
        else:
            five_utr = [(chrom, max(exon[1], cds[-1][2] + 1), exon[2], strand) for exon in exons if exon[1] > cds[-1][2]]
            cds_region = [(chrom, max(exon[1], cds[0][1]), min(exon[2], cds[-1][2]), strand) for exon in exons if exon[1] <= cds[-1][2] and exon[2] >= cds[0][1]]
            three_utr = [(chrom, exon[1], min(exon[2], cds[0][1] - 1), strand) for exon in exons if exon[2] < cds[0][1]]

        regions["5UTR"].extend(five_utr)
        regions["CDS"].extend(cds_region)
        regions["3UTR"].extend(three_utr)

    return regions

In [34]:
regions = parse_gtf(gtf_file)
regions

{'5UTR': [], 'CDS': [], '3UTR': []}

In [101]:
transcripts = {}

with open(gtf_file, 'r') as f:
    for line in f:
        if line.startswith("#"):
            continue
        fields = line.strip().split('\t')
        feature_type = fields[2]
        chrom = fields[0]
        start = int(fields[3])
        end = int(fields[4])
        strand = fields[6]
        attributes = fields[8]
        attr = {
            item.split('=')[0]: item.split('=')[1]
            for item in attributes.split(';') if '=' in item
        }
        transcript_id = attr_dict.get("ID")
        if transcript_id not in transcripts:
            transcripts[transcript_id] = {"exons": [], "cds": []}
            
        if feature_type == "exon" and transcript_id:
            transcripts[transcript_id]["exons"].append((chrom, start, end, strand))
        elif feature_type == "CDS" and transcript_id:
            transcripts[transcript_id]["cds"].append((chrom, start, end, strand))

x = list(transcripts.keys())
print (x)

['NR_110561.1_1']
